# Drifting y Quality

Un desafío común para los **data scientists** en el mundo real es que, a diferencia de las competencia de **Kaggle**, los datos cambian con el tiempo. Esto significa que los datos que recibiremos en el futuro no necesariamente reflejarán los que usamos para entrenar nuestros modelos. Esta homogeneidad es esencial, ya que los algoritmos aprenden patrones en función de las distribuciones observadas durante el entrenamiento. Si esas distribuciones cambian, la capacidad predictiva del modelo se deteriora.

Podemos identificar dos tipos principales de problemáticas asociadas a estos cambios:

- **Data Quality (DQ)**
- **Data Drift (DF)**

Comencemos con el primero: **Data Quality (DQ)**. Hay mucho que decir sobre este tema, pero mencionaremos brevemente que los datos provienen de procesos creados por seres humanos, quienes son propensos a cometer errores y hacer modificaciones. A lo largo del ciclo de vida de los modelos, es común que se enfrenten a este tipo de situaciones. Por ello, es fundamental implementar controles que garanticen que los datos a los que se aplican los modelos mantienen la misma estructura que los datos usados en el entrenamiento.

A continuación, veremos dos indicadores clave para monitorear la calidad de los datos, aunque no son los únicos a considerar.


In [1]:
import duckdb
import pandas as pd
import numpy as np
import seaborn as sns
import os
import polars as ps
import matplotlib.pyplot as plt

In [2]:
base_path = '/home/fedepicado/'
modelos_path = base_path + 'buckets/b1/modelos/'
db_path = base_path + 'buckets/b1/db/'
dataset_path = base_path + 'buckets/b1/datasets/'
exp_path = base_path + 'buckets/b1/exp/'
dataset_file = 'competencia_02.parquet'
full_path = dataset_path + dataset_file

In [3]:
# Verifica si el archivo existe
if os.path.exists(full_path):
    print("Archivo encontrado:", full_path)
else:
    print("Archivo no encontrado:", full_path)

Archivo encontrado: /home/fedepicado/buckets/b1/datasets/competencia_02.parquet


In [4]:
# Leo el archivo
data = pd.read_parquet(dataset_path + dataset_file)

In [5]:
df=data.copy()

In [6]:
df=df.sort_values(by=['foto_mes', 'numero_de_cliente'])

In [8]:
df.groupby("foto_mes")['clase_ternaria'].value_counts().reset_index().head(60)# tengo problemas con el primer mes

,foto_mes,clase_ternaria,count
0,201901,CONTINUA,1678
1,201901,BAJA+1,9
2,201901,BAJA+2,3
3,201902,CONTINUA,124391
4,201902,BAJA+1,720
5,201902,BAJA+2,688
6,201903,CONTINUA,124988
7,201903,BAJA+2,760
8,201903,BAJA+1,688
9,201904,CONTINUA,125658


## NaN por foto_mes para todas las variables

In [9]:
def calcular_porcentaje_nan_por_mes(dataset):
    # Crear una lista de las variables que quiero analizar los NaN
    variables = dataset.columns.difference(['numero_de_cliente', 'foto_mes', 'clase_ternaria'])

    # Inicializar una lista para almacenar los resultados
    resultados = []

    # Agrupar el dataset por 'foto_mes' y calcular el ratio de NaN para cada columna
    for mes, datos_mes in dataset.groupby('foto_mes'):
        # Calcular el porcentaje de NaN para cada columna en este mes
        porcentaje_nan = datos_mes[variables].isna().mean() * 100
        # Crear un DataFrame temporal para almacenar los resultados
        resultado_mes = pd.DataFrame({
            'foto_mes': mes,
            'campo': porcentaje_nan.index,
            'porcentaje_nan': porcentaje_nan.values
        })
        # Añadir los resultados del mes a la lista
        resultados.append(resultado_mes)

    # Concatenar todos los resultados en un único DataFrame
    tabla_resultados_nan = pd.concat(resultados, ignore_index=True)

    return tabla_resultados_nan

In [10]:
pd.options.display.max_columns = 152

In [11]:
# Llamada a la función
tabla_porcentajes_nan_por_mes = calcular_porcentaje_nan_por_mes(df)
tabla_pivoteada_nan = tabla_porcentajes_nan_por_mes.pivot(index='campo', columns='foto_mes', values='porcentaje_nan')
tabla_pivoteada_nan['total_nan'] = tabla_pivoteada_nan.sum(axis=1)/6

# Ordenar la tabla por la columna 'total_nan' en orden descendente
tabla_ordenada_nan = tabla_pivoteada_nan.sort_values(by='total_nan', ascending=False)
tabla_ordenada_nan.iloc[0:50].style.background_gradient(cmap='YlOrRd', axis=1)

foto_mes,201901,201902,201903,201904,201905,201906,201907,201908,201909,201910,201911,201912,202001,202002,202003,202004,202005,202006,202007,202008,202009,202010,202011,202012,202101,202102,202103,202104,202105,202106,202107,202108,total_nan
campo,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Master_Finiciomora,99.512633,99.349756,99.355405,99.534631,99.477514,99.492205,99.659588,99.477628,99.594234,99.588827,99.387742,99.517279,99.592959,99.394327,99.466376,99.655039,99.384508,99.726842,99.695693,99.633257,99.827620,99.434675,99.562031,99.726979,99.631541,99.604663,99.310261,99.372905,99.493388,99.596060,99.548900,99.634313,530.873296
Visa_Finiciomora,99.137489,98.949912,94.908096,99.294466,99.196296,95.861007,99.472935,99.145209,95.627410,99.310809,98.976685,99.249970,99.337344,99.144172,99.140979,99.536271,96.761888,99.531729,99.576924,97.152008,99.800469,99.122274,97.283846,99.702834,97.367706,99.394390,99.036564,99.140106,97.336338,99.390451,99.409029,97.495195,525.631800
Master_cconsumos,56.038380,56.582326,57.085798,57.158493,57.411542,58.343009,58.169885,58.783091,59.666155,59.378704,59.675337,60.267594,60.653210,61.445595,62.099280,62.014919,63.306470,63.385732,62.864333,62.536770,62.186890,62.325410,61.924077,61.059520,60.885290,60.002705,59.844824,59.780608,59.299733,58.459691,57.903628,58.356403,319.815900
Master_cadelantosefectivo,56.038380,56.582326,57.085798,57.158493,57.411542,58.343009,58.169885,58.783091,59.666155,59.378704,59.675337,60.267594,60.653210,61.445595,62.099280,62.014919,63.306470,63.385732,62.864333,62.536770,62.186890,62.325410,61.924077,61.059520,60.885290,60.002705,59.844824,59.780608,59.299733,58.459691,57.903628,58.356403,319.815900
Master_mconsumosdolares,56.038380,56.582326,57.085798,57.158493,57.411542,58.343009,58.169885,58.783091,59.666155,59.378704,59.675337,60.267594,60.653210,61.445595,62.099280,62.014919,63.306470,63.385732,62.864333,62.536770,62.186890,62.325410,61.924077,61.059520,60.885290,60.002705,59.844824,59.780608,59.299733,58.459691,57.903628,58.356403,319.815900
Master_mconsumospesos,56.038380,56.582326,57.085798,57.158493,57.411542,58.343009,58.169885,58.783091,59.666155,59.378704,59.675337,60.267594,60.653210,61.445595,62.099280,62.014919,63.306470,63.385732,62.864333,62.536770,62.186890,62.325410,61.924077,61.059520,60.885290,60.002705,59.844824,59.780608,59.299733,58.459691,57.903628,58.356403,319.815900
Master_madelantopesos,56.038380,56.582326,57.085798,57.158493,57.411542,58.343009,58.169885,58.783091,59.666155,59.378704,59.675337,60.267594,60.653210,61.445595,62.099280,62.014919,63.306470,63.385732,62.864333,62.536770,62.186890,62.325410,61.924077,61.059520,60.885290,60.002705,59.844824,59.780608,59.299733,58.459691,57.903628,58.356403,319.815900
Master_madelantodolares,56.038380,56.582326,57.085798,57.158493,57.411542,58.343009,58.169885,58.783091,59.666155,59.378704,59.675337,60.267594,60.653210,61.445595,62.099280,62.014919,63.306470,63.385732,62.864333,62.536770,62.186890,62.325410,61.924077,61.059520,60.885290,60.002705,59.844824,59.780608,59.299733,58.459691,57.903628,58.356403,319.815900
Master_mpagosdolares,56.038380,56.582326,57.085798,57.158493,57.411542,58.343009,58.169885,58.783091,59.666155,59.378704,59.675337,60.267594,60.653210,61.445595,62.099280,62.014919,63.306470,63.385732,62.864333,62.536770,62.186890,62.325410,61.924077,61.059520,60.885290,60.002705,59.844824,59.780608,59.299733,58.459691,57.903628,58.356403,319.815900


In [12]:
lista_meses=[201901,201902,201903,201904,201905,201906]
lista_columnas=["cmobile_app_trx","tmobile_app"]

In [13]:
def asignar_cero(df, lista_meses, lista_columnas):
    for mes in lista_meses:
        filtro_mes = df['foto_mes'] == mes
        for columna in lista_columnas:
            df.loc[filtro_mes, columna] = df.loc[filtro_mes, columna].fillna(0)
    return df

In [14]:
df=asignar_cero(df,lista_meses,lista_columnas)

In [17]:
df["internet"].value_counts()

internet
1    2595829
0    2139764
Name: count, dtype: int64

In [16]:
df.loc[df["internet"] > 1, "internet"] = 1

In [18]:
#Elimino las dos que tienen muchos NaN y 2 con drift
col_eliminar_nan=["Master_Finiciomora","Visa_Finiciomora","Visa_fultimo_cierre","Master_fultimo_cierre", "cprestamos_personales","mprestamos_personales"]
df.drop(columns=col_eliminar_nan, axis=1,inplace=True)

In [19]:
# Llamada a la función
tabla_porcentajes_nan_por_mes = calcular_porcentaje_nan_por_mes(df)
tabla_pivoteada_nan = tabla_porcentajes_nan_por_mes.pivot(index='campo', columns='foto_mes', values='porcentaje_nan')
tabla_pivoteada_nan['total_nan'] = tabla_pivoteada_nan.sum(axis=1)/6

# Ordenar la tabla por la columna 'total_nan' en orden descendente
tabla_ordenada_nan = tabla_pivoteada_nan.sort_values(by='total_nan', ascending=False)
tabla_ordenada_nan.iloc[0:50].style.background_gradient(cmap='YlOrRd', axis=1)

foto_mes,201901,201902,201903,201904,201905,201906,201907,201908,201909,201910,201911,201912,202001,202002,202003,202004,202005,202006,202007,202008,202009,202010,202011,202012,202101,202102,202103,202104,202105,202106,202107,202108,total_nan
campo,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Master_cadelantosefectivo,56.038380,56.582326,57.085798,57.158493,57.411542,58.343009,58.169885,58.783091,59.666155,59.378704,59.675337,60.267594,60.653210,61.445595,62.099280,62.014919,63.306470,63.385732,62.864333,62.536770,62.186890,62.325410,61.924077,61.059520,60.885290,60.002705,59.844824,59.780608,59.299733,58.459691,57.903628,58.356403,319.815900
Master_cconsumos,56.038380,56.582326,57.085798,57.158493,57.411542,58.343009,58.169885,58.783091,59.666155,59.378704,59.675337,60.267594,60.653210,61.445595,62.099280,62.014919,63.306470,63.385732,62.864333,62.536770,62.186890,62.325410,61.924077,61.059520,60.885290,60.002705,59.844824,59.780608,59.299733,58.459691,57.903628,58.356403,319.815900
Master_mconsumospesos,56.038380,56.582326,57.085798,57.158493,57.411542,58.343009,58.169885,58.783091,59.666155,59.378704,59.675337,60.267594,60.653210,61.445595,62.099280,62.014919,63.306470,63.385732,62.864333,62.536770,62.186890,62.325410,61.924077,61.059520,60.885290,60.002705,59.844824,59.780608,59.299733,58.459691,57.903628,58.356403,319.815900
Master_madelantodolares,56.038380,56.582326,57.085798,57.158493,57.411542,58.343009,58.169885,58.783091,59.666155,59.378704,59.675337,60.267594,60.653210,61.445595,62.099280,62.014919,63.306470,63.385732,62.864333,62.536770,62.186890,62.325410,61.924077,61.059520,60.885290,60.002705,59.844824,59.780608,59.299733,58.459691,57.903628,58.356403,319.815900
Master_mconsumosdolares,56.038380,56.582326,57.085798,57.158493,57.411542,58.343009,58.169885,58.783091,59.666155,59.378704,59.675337,60.267594,60.653210,61.445595,62.099280,62.014919,63.306470,63.385732,62.864333,62.536770,62.186890,62.325410,61.924077,61.059520,60.885290,60.002705,59.844824,59.780608,59.299733,58.459691,57.903628,58.356403,319.815900
Master_madelantopesos,56.038380,56.582326,57.085798,57.158493,57.411542,58.343009,58.169885,58.783091,59.666155,59.378704,59.675337,60.267594,60.653210,61.445595,62.099280,62.014919,63.306470,63.385732,62.864333,62.536770,62.186890,62.325410,61.924077,61.059520,60.885290,60.002705,59.844824,59.780608,59.299733,58.459691,57.903628,58.356403,319.815900
Master_mpagosdolares,56.038380,56.582326,57.085798,57.158493,57.411542,58.343009,58.169885,58.783091,59.666155,59.378704,59.675337,60.267594,60.653210,61.445595,62.099280,62.014919,63.306470,63.385732,62.864333,62.536770,62.186890,62.325410,61.924077,61.059520,60.885290,60.002705,59.844824,59.780608,59.299733,58.459691,57.903628,58.356403,319.815900
Master_mconsumototal,56.038380,56.582326,57.085798,57.158493,57.411542,58.343009,58.169885,58.783091,59.666155,59.378704,59.675337,60.267594,60.653210,61.445595,62.099280,62.014919,63.306470,63.385732,62.864333,62.536770,62.186890,62.325410,61.924077,61.059520,60.885290,60.002705,59.844824,59.780608,59.299733,58.459691,57.903628,58.356403,319.815900
Master_mpagospesos,56.038380,56.582326,57.085798,57.158493,57.411542,58.343009,58.169885,58.783091,59.666155,59.378704,59.675337,60.267594,60.653210,61.445595,62.099280,62.014919,63.306470,63.385732,62.864333,62.536770,62.186890,62.325410,61.924077,61.059520,60.885290,60.002705,59.844824,59.780608,59.299733,58.459691,57.903628,58.356403,319.815900


In [20]:
df.shape

(4735593, 149)

## Sigo aca

In [21]:
def calcular_porcentaje_ceros_por_mes(dataset):
    # Crear una lista de las variables que quiero analizar los ceros
    variables = dataset.columns.difference(['numero_de_cliente', 'foto_mes', 'clase_ternaria'])

    # Inicializar una lista para almacenar los resultados
    resultados = []

    # Agrupar el dataset por 'foto_mes' y calcular el ratio de ceros para cada columna
    for mes, datos_mes in dataset.groupby('foto_mes'):
        # Calcular el porcentaje de ceros para cada columna en este mes
        porcentaje_ceros = (datos_mes[variables] == 0).mean() * 100
        # Crear un DataFrame temporal para almacenar los resultados
        resultado_mes = pd.DataFrame({
            'foto_mes': mes,
            'campo': porcentaje_ceros.index,
            'porcentaje_ceros': porcentaje_ceros.values
        })
        # Añadir los resultados del mes a la lista
        resultados.append(resultado_mes)

    # Concatenar todos los resultados en un único DataFrame
    tabla_resultados = pd.concat(resultados, ignore_index=True)

    return tabla_resultados

In [22]:
# Llamada a la función
tabla_porcentajes_ceros_por_mes = calcular_porcentaje_ceros_por_mes(df)
tabla_pivoteada = tabla_porcentajes_ceros_por_mes.pivot(index='campo', columns='foto_mes', values='porcentaje_ceros')
tabla_pivoteada['total_zero'] = tabla_pivoteada.sum(axis=1)/6

# Ordenar la tabla por la columna 'total_zero' en orden descendente
tabla_pivoteada_zero = tabla_pivoteada.sort_values(by='total_zero', ascending=False)


In [23]:
tabla_pivoteada_zero.iloc[0:50].style.background_gradient(cmap='YlOrRd', axis=1)

foto_mes,201901,201902,201903,201904,201905,201906,201907,201908,201909,201910,201911,201912,202001,202002,202003,202004,202005,202006,202007,202008,202009,202010,202011,202012,202101,202102,202103,202104,202105,202106,202107,202108,total_zero
campo,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
mcuenta_corriente_adicional,99.920643,99.919713,99.919327,99.920470,99.922450,99.924140,99.924268,99.928391,99.926292,99.929032,99.926442,99.928907,99.933318,99.934742,99.935055,99.933944,99.934550,99.935613,99.935800,99.935693,99.937489,99.939273,99.937789,99.938090,99.937047,99.937902,99.937074,99.937839,99.939255,99.940561,99.941872,99.941974,532.972492
mpayroll2,99.978357,99.677263,99.686798,99.656682,99.680399,99.695787,99.687892,99.677380,99.746117,99.737347,99.756251,99.660887,99.729797,99.760042,99.859396,99.895911,99.894223,99.905695,99.910762,99.887303,99.893289,99.891067,99.906061,99.852036,99.875946,99.889330,99.904084,99.909806,99.907668,99.871419,99.929156,99.935929,532.375013
cpayroll2_trx,99.978357,99.677263,99.686798,99.656682,99.680399,99.695787,99.687892,99.677380,99.746117,99.735883,99.756251,99.660887,99.729797,99.760042,99.859396,99.895911,99.894223,99.905695,99.910762,99.887303,99.893289,99.891067,99.906061,99.852036,99.875946,99.889330,99.904084,99.909806,99.907668,99.871419,99.929156,99.935929,532.374769
ccheques_emitidos_rechazados,99.621649,99.725753,99.686798,99.244071,99.452448,99.676436,99.651173,99.701502,99.691023,99.675158,99.712260,99.741222,99.735354,99.753244,99.676612,99.405493,99.617879,100.000000,99.777869,99.812171,99.802363,99.825331,99.824564,99.818605,99.777196,99.822313,99.829551,99.834847,99.831737,99.833815,99.840753,99.795094,531.865714
mcheques_emitidos_rechazados,99.621649,99.725753,99.686798,99.244071,99.452448,99.676436,99.651173,99.701502,99.691023,99.675158,99.712260,99.741222,99.735354,99.753244,99.676612,99.405493,99.617879,100.000000,99.777869,99.812171,99.802363,99.825331,99.824564,99.818605,99.777196,99.822313,99.829551,99.834847,99.831737,99.833815,99.840753,99.795094,531.865714
mcheques_depositados_rechazados,99.525459,99.611285,99.600588,99.181864,99.243297,99.573483,99.518834,99.621600,99.589767,99.561756,99.668991,99.675816,99.683259,99.702941,99.433568,99.113911,99.584162,100.000000,99.811253,99.815992,99.809309,99.824705,99.815854,99.808700,99.781517,99.835225,99.829551,99.803157,99.811691,99.793178,99.821377,99.840427,531.548753
ccheques_depositados_rechazados,99.525459,99.611285,99.600588,99.181864,99.243297,99.573483,99.518834,99.621600,99.589767,99.561756,99.668991,99.675816,99.683259,99.702941,99.433568,99.113911,99.584162,100.000000,99.811253,99.815992,99.809309,99.824705,99.815854,99.808700,99.781517,99.835225,99.829551,99.803157,99.811691,99.793178,99.821377,99.840427,531.548753
mplazo_fijo_pesos,99.345101,99.351346,99.323769,99.373209,99.372547,99.375319,99.370429,99.430139,99.571899,99.612239,99.631491,99.620364,99.636720,99.617291,99.641795,99.622344,99.623168,99.598717,99.580776,99.566402,99.565577,99.638142,99.660325,99.634734,99.638330,99.626797,99.610838,99.608142,99.599691,99.600306,99.574937,99.553922,530.929468
cliente_vip,99.250513,99.255161,99.254959,99.260607,99.264447,99.272367,99.270218,99.270337,99.272600,99.282276,99.292550,99.302579,99.657558,99.664195,99.667238,99.667716,99.670107,99.674812,99.677718,99.677826,99.677971,99.680713,99.677744,99.676213,99.678447,99.728859,99.729358,99.728198,99.727863,99.434727,99.722074,99.717726,530.797613


In [36]:
tabla_pivoteada_zero.iloc[50:100].style.background_gradient(cmap='YlOrRd', axis=1)

foto_mes,201901,201902,201903,201904,201905,201906,201907,201908,201909,201910,201911,201912,202001,202002,202003,202004,202005,202006,202007,202008,202009,202010,202011,202012,202101,202102,202103,202104,202105,202106,202107,202108,total_zero
campo,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Visa_cadelantosefectivo,86.647909,86.406092,86.225442,86.068065,86.044071,85.591318,86.036994,85.822831,85.595694,85.764036,85.738496,85.684731,85.638970,85.631742,85.308257,85.149327,84.438818,84.366891,84.667189,84.910033,85.287711,85.134382,85.242189,85.620891,85.736857,86.003345,86.028653,86.181973,86.322081,86.658458,87.377688,86.919283,457.375070
mforex_sell,86.211043,89.850476,89.718909,87.484645,88.473981,85.659437,83.698479,85.839414,82.882648,74.866478,83.598116,74.698033,94.608449,95.967616,95.055438,85.147993,73.854463,68.992631,64.132277,63.134638,68.735438,87.707458,89.380498,90.681376,92.093244,94.125893,95.777866,96.319703,96.321292,94.737257,92.030978,92.252874,457.339840
cforex_sell,86.211043,89.850476,89.718909,87.484645,88.473981,85.659437,83.698479,85.839414,82.882648,74.866478,83.598116,74.698033,94.608449,95.967616,95.055438,85.147993,73.854463,68.992631,64.132277,63.134638,68.735438,87.707458,89.380498,90.681376,92.093244,94.125893,95.777866,96.319703,96.321292,94.737257,92.030978,92.252874,457.339840
cseguro_vida,78.515775,78.581706,78.661141,78.829254,79.006572,79.205951,79.291484,79.778237,79.976771,80.252703,86.065899,86.213663,86.310657,86.435228,86.518118,86.608573,86.695844,86.693939,86.594463,86.603675,86.552462,86.447840,88.521357,88.551688,88.504931,88.445458,88.429606,88.428302,88.446936,88.416143,88.442768,88.435222,452.410395
Master_msaldodolares,80.656022,81.162807,81.408776,81.151375,81.798385,81.796015,81.481595,82.327534,82.824575,82.391244,83.137300,83.228471,83.960796,84.644719,85.136854,85.363510,85.583197,86.364848,86.103978,86.322250,86.129405,86.114154,86.185938,85.621510,85.946700,85.810902,86.244922,86.201475,86.186013,85.892428,86.693470,86.067625,449.989799
cforex,81.136976,84.036439,83.363124,81.971086,82.901323,81.309120,79.751232,80.002111,78.725970,71.704394,79.670001,72.758618,93.040718,94.767825,94.020997,84.382673,73.502092,68.692157,63.868416,62.792090,68.386889,87.368138,89.043312,90.373067,91.811808,93.828314,95.309283,95.936376,96.035183,94.444916,91.839033,91.993569,446.461209
cseguro_accidentes_personales,80.739387,80.820992,80.978519,81.084444,81.252399,81.453873,81.519078,81.634053,81.780753,81.964706,82.137062,82.297154,82.409041,82.626488,82.813546,83.008834,83.341377,83.707408,83.913485,84.283513,84.631025,84.910255,85.120255,85.311343,85.449249,85.631986,85.863091,86.060089,86.255262,86.429195,86.593562,86.771195,445.465436
mttarjeta_master_debitos_automaticos,81.314929,81.568216,81.610459,81.304136,81.987952,81.898193,81.500719,81.940089,82.794050,81.829356,82.750041,83.313783,82.615340,83.081253,83.015078,83.083565,84.083802,83.695051,85.686038,84.794152,83.841107,84.353695,84.880118,84.051484,84.086505,83.567994,85.929682,83.974648,84.886680,83.776292,84.918742,84.209572,444.390454
ctarjeta_master_debitos_automaticos,81.314929,81.568216,81.610459,81.304136,81.987952,81.898193,81.500719,81.940089,82.794050,81.829356,82.750041,83.313783,82.615340,83.081253,83.015078,83.083565,84.083802,83.695051,85.686038,84.794152,83.841107,84.353695,84.880118,84.051484,84.086505,83.567994,85.929682,83.974648,84.886680,83.776292,84.918742,84.209572,444.390454


In [24]:
tabla_pivoteada_zero.iloc[100:153].style.background_gradient(cmap='YlOrRd', axis=1)

foto_mes,201901,201902,201903,201904,201905,201906,201907,201908,201909,201910,201911,201912,202001,202002,202003,202004,202005,202006,202007,202008,202009,202010,202011,202012,202101,202102,202103,202104,202105,202106,202107,202108,total_zero
campo,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
mautoservicio,35.809446,35.859586,35.875067,35.902706,35.815728,36.161813,34.736544,35.210758,34.944235,34.056423,34.657849,32.930237,33.632941,34.150188,35.022363,39.134728,37.966826,100.000000,35.112735,34.514638,33.843949,32.380690,31.912008,29.453463,30.301310,30.629711,29.163332,29.783046,30.100897,29.163129,28.214614,28.139167,189.096687
ctarjeta_debito_transacciones,35.809446,35.859586,35.875067,35.902706,35.815728,36.161813,34.736544,35.210758,34.944235,34.056423,34.657849,32.930237,33.632941,34.120278,35.022363,39.134728,37.966826,100.000000,35.112735,34.514638,33.843949,32.380690,31.912008,29.453463,30.301310,30.629711,29.163332,29.783046,30.100897,29.163129,28.214614,28.139167,189.091702
chomebanking_transacciones,40.335225,32.998672,36.176405,41.082396,47.760832,48.236651,43.809094,44.237321,44.970740,100.000000,19.574953,19.164516,20.157537,20.817897,20.771847,18.511797,16.983228,100.000000,16.043502,16.216939,16.247924,16.342476,17.314703,16.779342,17.296607,16.976747,15.635519,16.004022,15.785765,16.109076,15.519643,15.439248,157.216771
thomebanking,20.885437,22.095565,21.691607,22.421179,21.168112,22.274085,22.428934,22.468040,23.456974,23.027904,24.285519,23.736501,25.221927,26.498719,26.289536,26.704788,21.734618,100.000000,21.958219,22.307683,22.535060,24.373478,26.304886,26.523284,26.724106,27.008964,25.076824,26.305686,26.244206,27.385429,26.943664,26.346393,142.071221
mtarjeta_visa_consumo,14.378126,14.768003,14.942738,14.778418,14.999334,15.284164,14.711912,15.174426,15.369954,15.040752,15.206213,15.197532,15.305697,16.585661,15.953828,16.371304,16.627551,100.000000,16.672017,16.327726,15.952416,16.005660,15.993953,15.512054,15.514794,15.102124,15.385649,15.100859,15.058649,14.487251,14.493921,14.387519,96.115034
ctarjeta_visa_transacciones,14.378126,14.768003,14.942738,14.778418,14.999334,15.284164,14.711912,15.174426,15.369954,15.040752,15.206213,15.197532,15.305697,16.585661,15.953828,16.371304,16.627551,100.000000,16.672017,16.327726,15.952416,16.005660,15.993953,15.512054,15.514794,15.102124,15.385649,15.100859,15.058649,14.487251,14.493921,14.387519,96.115034
Visa_mpagominimo,14.604976,14.396776,14.115442,14.394942,14.527765,16.770393,14.770050,14.546524,14.851021,14.655917,13.767515,15.427162,15.081339,16.052043,16.271861,16.510089,19.511969,20.687839,18.567833,17.660355,16.827576,15.795932,15.318958,17.048649,14.153901,14.601035,13.690320,13.365226,13.212006,13.165652,12.923852,12.079762,81.559113
mactivos_margen,3.621585,3.930079,3.997279,4.108003,100.000000,3.912963,4.479667,4.486522,4.394925,100.000000,4.687489,4.523642,4.716391,4.889572,5.029594,5.724218,5.429027,100.000000,6.063018,5.403736,5.724533,5.454170,4.775976,5.035103,4.484465,4.844263,4.488499,4.101408,4.064438,4.371770,4.354776,3.901065,72.499696
mcomisiones,2.306175,2.619258,2.655889,2.667801,100.000000,2.939173,2.668982,2.813876,3.471716,100.000000,2.848551,2.728546,2.876374,3.250651,3.299499,3.402904,3.356450,100.000000,3.242726,3.378370,3.178612,3.210397,3.067027,2.847839,2.656364,2.720018,2.328252,2.566275,2.668521,2.514617,2.908835,2.960554,64.025709


In [25]:
# Sample function to interpolate zeros with values from adjacent months
def imputar_valores(df, columnas, meses_a_imputar):
    # Sort the DataFrame by client and month to ensure correct interpolation
    df = df.sort_values(['numero_de_cliente', 'foto_mes'])
    
    for col in columnas:
        # Replace zeros in the specified months with NaNs temporarily
        df.loc[(df['foto_mes'].isin(meses_a_imputar)) & (df[col] == 0), col] = None
        
        # Apply interpolation per client group, filling the NaNs created above
        df[col] = df.groupby('numero_de_cliente')[col].transform(
            lambda x: x.interpolate(method='linear', limit_direction='both')
        )
        
    return df

In [26]:
col_var_impu=["mactivos_margen","mcomisiones","mcomisiones_otras","ccomisiones_otras","mpasivos_margen","mrentabilidad","mrentabilidad_annual"]
meses=[201905, 201910]
df = imputar_valores(df, col_var_impu, meses)

In [27]:
dataset_file1 = 'competencia_02_DQ.parquet'
full_path1 = dataset_path + dataset_file1

# Guarda el DataFrame en formato Parquet
df.to_parquet(full_path1, engine='pyarrow') 

## Cuando entreno con mas meses, sin agregar ft me da menor ganancia que entrenando con solo dos meses, ya me empieza a jugar la inflacion... 18/11/2024

In [28]:
meses=[201901, 201902, 201903, 201904, 201905, 201906, 201907, 
       201908, 201909, 201910, 201911, 201912, 202001, 202002,
       202003, 202004, 202005, 202006, 202007, 202008, 202009,
       202010, 202011, 202012, 202101, 202102, 202103, 202104, 
       202105, 202106, 202107, 202108] #202109, 202110,202111,202112

IPC = [
    189.6, 196.8, 206.0, 213.1, 219.6, 225.5, 230.5, 239.6, 253.7, 262.1, 
    273.2, 283.4, 289.8, 295.7, 305.6, 310.1, 314.9, 322.0, 328.2, 337.1, 
    346.6, 359.7, 371.0, 385.9, 401.5, 415.9, 435.9, 453.7, 468.7, 483.6, 
    498.1, 510.4] #528.5, 547.1, 560.9, 582.5


In [4]:
dataset_file = 'competencia_02_DQ.parquet'
full_path = dataset_path + dataset_file

In [5]:
df = pd.read_parquet(dataset_path + dataset_file)

In [29]:
df.shape

(4735593, 149)

In [30]:
df.head(5)

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,mactivos_margen,mpasivos_margen,cproductos,tcuentas,ccuenta_corriente,mcuenta_corriente_adicional,mcuenta_corriente,ccaja_ahorro,mcaja_ahorro,mcaja_ahorro_adicional,mcaja_ahorro_dolares,cdescubierto_preacordado,mcuentas_saldo,ctarjeta_debito,ctarjeta_debito_transacciones,mautoservicio,ctarjeta_visa,ctarjeta_visa_transacciones,mtarjeta_visa_consumo,ctarjeta_master,ctarjeta_master_transacciones,mtarjeta_master_consumo,cprestamos_prendarios,mprestamos_prendarios,cprestamos_hipotecarios,mprestamos_hipotecarios,cplazo_fijo,mplazo_fijo_dolares,mplazo_fijo_pesos,cinversion1,minversion1_pesos,minversion1_dolares,cinversion2,minversion2,cseguro_vida,cseguro_auto,cseguro_vivienda,cseguro_accidentes_personales,ccaja_seguridad,cpayroll_trx,mpayroll,mpayroll2,cpayroll2_trx,ccuenta_debitos_automaticos,mcuenta_debitos_automaticos,ctarjeta_visa_debitos_automaticos,mttarjeta_visa_debitos_automaticos,ctarjeta_master_debitos_automaticos,mttarjeta_master_debitos_automaticos,cpagodeservicios,mpagodeservicios,cpagomiscuentas,mpagomiscuentas,ccajeros_propios_descuentos,mcajeros_propios_descuentos,ctarjeta_visa_descuentos,mtarjeta_visa_descuentos,ctarjeta_master_descuentos,mtarjeta_master_descuentos,ccomisiones_mantenimiento,mcomisiones_mantenimiento,ccomisiones_otras,mcomisiones_otras,cforex,cforex_buy,mforex_buy,cforex_sell,mforex_sell,ctransferencias_recibidas,mtransferencias_recibidas,ctransferencias_emitidas,mtransferencias_emitidas,cextraccion_autoservicio,mextraccion_autoservicio,ccheques_depositados,mcheques_depositados,ccheques_emitidos,mcheques_emitidos,ccheques_depositados_rechazados,mcheques_depositados_rechazados,ccheques_emitidos_rechazados,mcheques_emitidos_rechazados,tcallcenter,ccallcenter_transacciones,thomebanking,chomebanking_transacciones,ccajas_transacciones,ccajas_consultas,ccajas_depositos,ccajas_extracciones,ccajas_otras,catm_trx,matm,catm_trx_other,matm_other,ctrx_quarter,tmobile_app,cmobile_app_trx,Master_delinquency,Master_status,Master_mfinanciacion_limite,Master_Fvencimiento,Master_msaldototal,Master_msaldopesos,Master_msaldodolares,Master_mconsumospesos,Master_mconsumosdolares,Master_mlimitecompra,Master_madelantopesos,Master_madelantodolares,Master_mpagado,Master_mpagospesos,Master_mpagosdolares,Master_fechaalta,Master_mconsumototal,Master_cconsumos,Master_cadelantosefectivo,Master_mpagominimo,Visa_delinquency,Visa_status,Visa_mfinanciacion_limite,Visa_Fvencimiento,Visa_msaldototal,Visa_msaldopesos,Visa_msaldodolares,Visa_mconsumospesos,Visa_mconsumosdolares,Visa_mlimitecompra,Visa_madelantopesos,Visa_madelantodolares,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
928934,249221109,201901,1,0,1,59,276,7597.550,47433.580,5654.59,-915.570,2571.970,9,1,1,0.0,-317.98,2,24068.63,0.0,268675.63,1,280992.59,3,0,0.0,1,23,31249.45,1,5,9333.88,0,0.0,0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0,0,1,1,0,0,0.0,0.0,0,5,15333.25,6,19820.48,4,8786.59,0,0.0,6,14612.56,0,0.0,0,0.0,0,0.0,1,1199.97,18.0,5654.59,0,0,0.00,0,0.0,0,0.0,0,0.0,0,0.0,1,5161.2,0,0.0,0,0.0,0,0.0,0,0,1,71,1,1,4,2,0,0,0.0,0,0.0,164,0.0,0.0,0.0,0.0,209004.84,-1307.0,9333.88,9333.88,0.0,8786.59,0.0,232227.60,0.0,0.0,8815.09,-9635.71,0.0,6755.0,8786.59,4.0,0.0,398.82,0.0,0.0,274901.32,-1276.0,26663.38,31176.66,99.48,24336.99,4.28,305484.53,0.0,0.0,0.0,-44919.57,3.23,7136.0,24336.99,13.0,0.0,1466.25,CONTINUA
928935,249221109,201902,1,0,1,59,277,3800.930,46605.810,2104.71,-600.050,2046.100,9,1,1,0.0,0.00,2,23798.48,0.0,282828.56,1,315387.19,3,0,0.0,1,17,24360.67,1,5,9977.93,0,0.0,0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0,0,1,1,0,0,0.0,0.0,0,4,10504.27,7,18915.28,4,9430.63,0,0.0,5,11583.57,0,0.0,0,0.0,0,0.0,1,1199.97,14.0,2104.71,0,0,0.00,0,0.0,0,0.0,0,0.0,0,0.0,1,3519.0,0,0.0,0,0.0,0,0.0,0,0,1,77,1,0,0,0,1,0,0.0,0,0.0,147,0.0,0.0,0.0,0.0,211831.4

In [31]:
## columnas monetarias
var_monetarias=["mrentabilidad","mrentabilidad_annual","mcomisiones","mactivos_margen","mpasivos_margen","mcuenta_corriente",
               "mcaja_ahorro","mcaja_ahorro_adicional","mcaja_ahorro_dolares","mcuentas_saldo","mautoservicio","mtarjeta_visa_consumo",
               "mtarjeta_master_consumo","mtarjeta_master_consumo","mprestamos_prendarios","mprestamos_hipotecarios","mplazo_fijo_dolares",
               "mplazo_fijo_pesos","minversion1_pesos","minversion1_dolares","minversion2","mpayroll","mpayroll2","mcuenta_debitos_automaticos",
               "mttarjeta_visa_debitos_automaticos","mttarjeta_master_debitos_automaticos","mpagomiscuentas","mcajeros_propios_descuentos",
               "mtarjeta_visa_descuentos","mtarjeta_master_descuentos","mcomisiones_mantenimiento","mcomisiones_otras","mforex_buy",
               "mtransferencias_recibidas","mtransferencias_emitidas","mextraccion_autoservicio","mcheques_depositados","mcheques_emitidos",
               "mcheques_depositados_rechazados","mcheques_emitidos_rechazados","matm","matm_other","Master_mfinanciacion_limite",
               "Master_msaldototal","Master_msaldopesos","Master_msaldodolares","Master_mconsumospesos","Master_mconsumosdolares",
               "Master_mlimitecompra","Master_madelantopesos","Master_madelantodolares","Master_mpagado","Master_mpagospesos","Master_mpagosdolares",
               "Master_mconsumototal","Master_mpagominimo","Visa_mfinanciacion_limite","Visa_msaldototal", "Visa_msaldopesos",
                "Visa_msaldodolares", "Visa_mconsumospesos", "Visa_mconsumosdolares", "Visa_mlimitecompra", "Visa_madelantopesos",
                "Visa_madelantodolares","Visa_mpagado", "Visa_mpagospesos", "Visa_mpagosdolares","Visa_mconsumototal", "Visa_mpagominimo"]

In [32]:
def ajustar_por_inflacion(df, columnas_monetarias, meses, ipc, ipc_base_mes=201901):
    """
    Ajusta variables monetarias por inflación utilizando un IPC base y listas de meses e IPC.
    
    Parámetros:
    -----------
    - df: pd.DataFrame
        DataFrame con las variables monetarias y la columna `foto_mes`.
    - columnas_monetarias: list
        Lista de columnas monetarias a ajustar.
    - meses: list
        Lista de valores `foto_mes` en orden cronológico (e.g., 201901, 201902, ...).
    - ipc: list
        Lista de valores IPC correspondientes a los meses.
    - ipc_base_mes: int
        `foto_mes` que se utilizará como base para ajustar los valores (default: 201901).

    Retorna:
    --------
    - pd.DataFrame:
        DataFrame original con las columnas ajustadas por inflación.
    """
    # Crear diccionario de mapeo entre meses e IPC
    mapeo_ipc = dict(zip(meses, ipc))
    
    # Obtener el IPC base
    ipc_base = mapeo_ipc[ipc_base_mes]
    
    # Agregar columna temporal con el IPC correspondiente al `foto_mes`
    df['IPC'] = df['foto_mes'].map(mapeo_ipc)
    
    # Ajustar las columnas monetarias por inflación
    for col in columnas_monetarias:
        df[col] = df[col] * ipc_base / df['IPC']
    
    # Eliminar columna temporal IPC
    df.drop(columns=['IPC'], inplace=True)
    
    return df

In [33]:
df=ajustar_por_inflacion(df, var_monetarias, meses, IPC, ipc_base_mes=201901)

In [34]:
df

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,mactivos_margen,mpasivos_margen,cproductos,tcuentas,ccuenta_corriente,mcuenta_corriente_adicional,mcuenta_corriente,ccaja_ahorro,mcaja_ahorro,mcaja_ahorro_adicional,mcaja_ahorro_dolares,cdescubierto_preacordado,mcuentas_saldo,ctarjeta_debito,ctarjeta_debito_transacciones,mautoservicio,ctarjeta_visa,ctarjeta_visa_transacciones,mtarjeta_visa_consumo,ctarjeta_master,ctarjeta_master_transacciones,mtarjeta_master_consumo,cprestamos_prendarios,mprestamos_prendarios,cprestamos_hipotecarios,mprestamos_hipotecarios,cplazo_fijo,mplazo_fijo_dolares,mplazo_fijo_pesos,cinversion1,minversion1_pesos,minversion1_dolares,cinversion2,minversion2,cseguro_vida,cseguro_auto,cseguro_vivienda,cseguro_accidentes_personales,ccaja_seguridad,cpayroll_trx,mpayroll,mpayroll2,cpayroll2_trx,ccuenta_debitos_automaticos,mcuenta_debitos_automaticos,ctarjeta_visa_debitos_automaticos,mttarjeta_visa_debitos_automaticos,ctarjeta_master_debitos_automaticos,mttarjeta_master_debitos_automaticos,cpagodeservicios,mpagodeservicios,cpagomiscuentas,mpagomiscuentas,ccajeros_propios_descuentos,mcajeros_propios_descuentos,ctarjeta_visa_descuentos,mtarjeta_visa_descuentos,ctarjeta_master_descuentos,mtarjeta_master_descuentos,ccomisiones_mantenimiento,mcomisiones_mantenimiento,ccomisiones_otras,mcomisiones_otras,cforex,cforex_buy,mforex_buy,cforex_sell,mforex_sell,ctransferencias_recibidas,mtransferencias_recibidas,ctransferencias_emitidas,mtransferencias_emitidas,cextraccion_autoservicio,mextraccion_autoservicio,ccheques_depositados,mcheques_depositados,ccheques_emitidos,mcheques_emitidos,ccheques_depositados_rechazados,mcheques_depositados_rechazados,ccheques_emitidos_rechazados,mcheques_emitidos_rechazados,tcallcenter,ccallcenter_transacciones,thomebanking,chomebanking_transacciones,ccajas_transacciones,ccajas_consultas,ccajas_depositos,ccajas_extracciones,ccajas_otras,catm_trx,matm,catm_trx_other,matm_other,ctrx_quarter,tmobile_app,cmobile_app_trx,Master_delinquency,Master_status,Master_mfinanciacion_limite,Master_Fvencimiento,Master_msaldototal,Master_msaldopesos,Master_msaldodolares,Master_mconsumospesos,Master_mconsumosdolares,Master_mlimitecompra,Master_madelantopesos,Master_madelantodolares,Master_mpagado,Master_mpagospesos,Master_mpagosdolares,Master_fechaalta,Master_mconsumototal,Master_cconsumos,Master_cadelantosefectivo,Master_mpagominimo,Visa_delinquency,Visa_status,Visa_mfinanciacion_limite,Visa_Fvencimiento,Visa_msaldototal,Visa_msaldopesos,Visa_msaldodolares,Visa_mconsumospesos,Visa_mconsumosdolares,Visa_mlimitecompra,Visa_madelantopesos,Visa_madelantodolares,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
928934,249221109,201901,1,0,1,59,276,7597.550000,47433.580000,5654.590000,-915.570000,2571.970000,9,1,1,0.0,-317.980000,2,24068.630000,0.0,268675.630000,1,280992.590000,3,0,0.000000,1,23,31249.450000,1,5,9333.880000,0,0.0,0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0,0,1,1,0,0,0.000000,0.0,0,5,15333.250000,6,19820.480000,4,8786.590000,0,0.0,6,14612.560000,0,0.0,0,0.0,0,0.0,1,1199.970000,18.0,5654.590000,0,0,0.000000,0,0.00,0,0.000000,0,0.0,0,0.0,1,5161.200000,0,0.0,0,0.0,0,0.0,0,0,1,71,1,1,4,2,0,0,0.0,0,0.0,164,0.0,0.0,0.0,0.0,209004.840000,-1307.0,9333.880000,9333.880000,0.0,8786.590000,0.0,232227.600000,0.0,0.0,8815.090000,-9635.710000,0.0,6755.0,8786.590000,4.0,0.0,398.820000,0.0,0.0,274901.320000,-1276.0,26663.380000,31176.660000,99.480000,24336.990000,4.280000,305484.530000,0.0,0.0,0.0,-44919.570000,3.230000,7136.0,24336.990000,13.0,0.0,1466.250000,CONTINUA
928935,249221109,201902,1,0,1,59,277,3661.871585,44900.719390,2027.708415,-578.096951,1971.242683,9,1,1,0.0,0.000000,2,22927.803902,0.0,272481.173659,1,303848.634268,3,0,0.000000,1,17,23469.425976,1,5,9261.192910,0,0.0,0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0,0,1,1,0,0,0.000

In [35]:
dataset_file1 = 'competencia_02_DQ_IPC.parquet'
full_path1 = dataset_path + dataset_file1

# Guarda el DataFrame en formato Parquet
df.to_parquet(full_path1, engine='pyarrow') 

In [43]:
aux = df[df['foto_mes'] == 202108]

In [44]:
aux.head(50)

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_madelantodolares,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
31,249221109,202108,1,0,0,62,307,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,8079.0,NaN,13.0,0.0,NaN,None
63,249221468,202108,1,0,0,53,44,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1336.0,NaN,26.0,0.0,NaN,None
95,249223005,202108,1,0,0,49,209,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3698.0,NaN,4.0,0.0,NaN,None
127,249228180,202108,1,0,0,66,327,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,9137.0,NaN,6.0,0.0,NaN,None
159,249232117,202108,1,0,0,80,380,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,8819.0,NaN,7.0,0.0,NaN,None
191,249236712,202108,1,0,0,45,32,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,953.0,NaN,0.0,0.0,NaN,None
221,249236857,202108,1,0,0,73,327,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,9671.0,NaN,2.0,0.0,NaN,None
253,249237079,202108,1,0,0,64,148,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4474.0,NaN,3.0,0.0,NaN,None
285,249237446,202108,1,0,0,54,284,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4165.0,NaN,2.0,0.0,NaN,None
317,249239632,202108,1,0,0,52,270,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5997.0,NaN,4.0,0.0,NaN,None


In [45]:
asd=calcular_porcentaje_nan_por_mes(aux)

In [46]:
tabla_pivoteada_nan = asd.pivot(index='campo', columns='foto_mes', values='porcentaje_nan')
tabla_pivoteada_nan['total_nan'] = tabla_pivoteada_nan.sum(axis=1)/6

# Ordenar la tabla por la columna 'total_nan' en orden descendente
tabla_ordenada_nan = tabla_pivoteada_nan.sort_values(by='total_nan', ascending=False)
tabla_ordenada_nan.iloc[0:50].style.background_gradient(cmap='YlOrRd', axis=1)

foto_mes,202108,total_nan
campo,,
Master_mconsumospesos,100.000000,16.666667
Master_madelantodolares,100.000000,16.666667
Master_mpagosdolares,100.000000,16.666667
Master_mpagospesos,100.000000,16.666667
Master_mconsumosdolares,100.000000,16.666667
Master_madelantopesos,100.000000,16.666667
Master_mconsumototal,100.000000,16.666667
Master_mpagado,100.000000,16.666667
Master_mlimitecompra,100.000000,16.666667
